In [234]:
import pandas as pd
import numpy as np

In [235]:
brand = pd.read_csv('./data/Extra_material_2.csv')

In [236]:
transaction = pd.read_csv('./data/Extra_material_3.csv', parse_dates=['date_id']) #parse_dates for standarization date format
transaction

,orderid,itemid,date_id,amount,item_price_usd,shopid
0,1316773501,1355135789,2019-05-16,1,5.483204,65948755
1,1316774684,1257902741,2019-05-16,1,0.387050,61364537
2,1316803330,2126770455,2019-05-16,1,0.344044,4980129
3,1317196434,2112999088,2019-05-16,1,6.704561,29089750
4,1318105348,1832131969,2019-05-16,1,5.354187,94372493
...,...,...,...,...,...,...
1048570,1282783920,2117708081,2019-05-05,1,2.451315,36050398
1048571,1282784581,2073505055,2019-05-05,1,1.311668,133389379
1048572,1282794723,1074306637,2019-05-05,1,1.161149,58327801
1048573,1282806746,1279044775,2019-05-05,1,30.060860,26210794


In [237]:
transaction_1 = pd.read_csv('./data/Extra_material_3.csv')
transaction_1

,orderid,itemid,date_id,amount,item_price_usd,shopid
0,1316773501,1355135789,16/5/2019,1,5.483204,65948755
1,1316774684,1257902741,16/5/2019,1,0.387050,61364537
2,1316803330,2126770455,16/5/2019,1,0.344044,4980129
3,1317196434,2112999088,16/5/2019,1,6.704561,29089750
4,1318105348,1832131969,16/5/2019,1,5.354187,94372493
...,...,...,...,...,...,...
1048570,1282783920,2117708081,5/5/2019,1,2.451315,36050398
1048571,1282784581,2073505055,5/5/2019,1,1.311668,133389379
1048572,1282794723,1074306637,5/5/2019,1,1.161149,58327801
1048573,1282806746,1279044775,5/5/2019,1,30.060860,26210794


In [238]:
all_brands = np.sort(brand.brand.unique())
all_brands

array(['3M', '3M Littmann', 'AHC', 'ASUS', 'Abbott', 'Acer', 'Adidas',
       'Anello', 'Anessa', 'Anessa, Senka, Za, Tsubaki, Ma Cherie',
       'Anker', 'Anlene', 'Anmum', 'Asiavina', 'Asics', 'Asus',
       'Asus Laptops', 'Asus Mobile', 'Aveeno', 'Baby Love', 'BabyLove',
       'Barbie', 'Bata', 'Bebelac', 'Beiersdorf', 'Bench', 'Bestmart',
       'Blackmores', 'Bosch', 'Bosch ', 'Bosch Power Tools', 'Bose',
       'Bourjois', 'Calvin Klein', 'Canon', 'Carefree', 'Casio', 'CeraVe',
       'Certainty', 'Clarks', 'Coca Cola', 'Coca-Cola', 'Coca-cola',
       'Colgate', 'Converse', 'Coolwinks', 'Coty',
       'Coty (Rimmel, Bourjois)', 'Crocs', 'Danone', 'Dell', 'Delsey',
       'Denizen', 'Dettol', 'Dr. Martens', 'Dugro', 'Dumex', 'Durex',
       'Dutch Lady', 'Dyson', 'Electrolux', 'Enfa', 'Enfamil', 'Ensure',
       'Etude House', 'Eucerin', 'Evian', 'Fernleaf', 'Fila',
       'Fisher-Price', 'Fitbit', 'Fititi, Certainity', 'Fjallraven',
       'Fonterra', 'Foremost', 'Fossil', 'Fr

In [239]:
brand = brand[brand['shop_type'] == 'Official Shop'].reset_index(drop=True) #only official Shop
brand['brand'][326]

'Anessa, Senka, Za, Tsubaki, Ma Cherie'

In [240]:
brand['shop_id'] = brand['shop_id'].astype(int) #float to int
brand

,shop_id,brand,shop_type
0,92567513,JBL,Official Shop
1,1657263,Sandisk,Official Shop
2,83401047,Revlon,Official Shop
3,4259661,Philips,Official Shop
4,4350106,Kimberly Clark,Official Shop
...,...,...,...
116167,13448324,Johnson & Johnson,Official Shop
116168,14171993,Sony,Official Shop
116169,14318452,Unilever,Official Shop
116170,14318520,Unilever,Official Shop


In [241]:
# filter only transaction from 10th May to 31st May
start_date = '2019-5-10'
end_date = '2019-5-31'

mask = (transaction['date_id'] >= start_date) & (transaction['date_id'] <= end_date)

transaction = transaction.loc[mask]

In [242]:
# drop duplicates row
transaction.drop_duplicates()
brand.drop_duplicates()

,shop_id,brand,shop_type
0,92567513,JBL,Official Shop
1,1657263,Sandisk,Official Shop
2,83401047,Revlon,Official Shop
3,4259661,Philips,Official Shop
4,4350106,Kimberly Clark,Official Shop
...,...,...,...
77548,56068753,Johnson's Baby,Official Shop
77549,56070310,Neutrogena,Official Shop
77550,56074501,Aveeno,Official Shop
77551,56075208,Listerine,Official Shop


In [243]:
# find possible mix product
find_mix = brand['brand']

In [244]:
mylist = []
for i in range(0, len(find_mix)):
    if(find_mix[i].find(',') != -1):
        mylist.append(i)

In [245]:
idx = np.array(mylist)

In [246]:
brand.drop(idx, inplace=True)

In [248]:
brand['brand']

0                       JBL
1                   Sandisk
2                    Revlon
3                   Philips
4            Kimberly Clark
                ...        
116167    Johnson & Johnson
116168                 Sony
116169             Unilever
116170             Unilever
116171              Huggies
Name: brand, Length: 114962, dtype: object

In [249]:
# filter only shop_id which is exist in transaction
brand = brand[brand['shop_id'].isin(transaction['shopid'])]

In [250]:
brand.head()

,shop_id,brand,shop_type
45,64903618,Harman Kardon,Official Shop
84,58542133,Sabina,Official Shop
95,75810249,Bosch,Official Shop
106,66829677,Nokia,Official Shop
120,82301700,The Body Shop,Official Shop


In [251]:
transaction.head()

,orderid,itemid,date_id,amount,item_price_usd,shopid
0,1316773501,1355135789,2019-05-16,1,5.483204,65948755
1,1316774684,1257902741,2019-05-16,1,0.387050,61364537
2,1316803330,2126770455,2019-05-16,1,0.344044,4980129
3,1317196434,2112999088,2019-05-16,1,6.704561,29089750
4,1318105348,1832131969,2019-05-16,1,5.354187,94372493


In [253]:
# there are 4 shop_id that have 2 brands
brand.groupby('shop_id')['brand'].nunique().sort_values(ascending=False)[:5]

shop_id
64903618    2
61963112    2
75810249    2
35284879    2
52679373    1
Name: brand, dtype: int64

In [254]:
transaction.rename(columns={'shopid': 'shop_id'}, inplace=True)

final_df = brand.merge(transaction, on='shop_id', how='left')

final_df.head()

C:\Users\asus\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,shop_id,brand,shop_type,orderid,itemid,date_id,amount,item_price_usd
0,64903618,Harman Kardon,Official Shop,1321879298,1921091854,2019-05-16,1,59.777675
1,58542133,Sabina,Official Shop,1321039647,1787516454,2019-05-16,1,7.246430
2,58542133,Sabina,Official Shop,1319191755,1618557460,2019-05-16,1,7.074408
3,58542133,Sabina,Official Shop,1375827952,1755241806,2019-05-29,1,9.203182
4,58542133,Sabina,Official Shop,1375901974,1648157592,2019-05-29,1,7.074408


In [255]:
final_df['gross_sales_revenue'] = final_df['amount'].values * final_df['item_price_usd'].values

In [257]:
final_df.sample(5)

,shop_id,brand,shop_type,orderid,itemid,date_id,amount,item_price_usd,gross_sales_revenue
421426,27495213,Senka,Official Shop,1333075005,2041024267,2019-05-19,1,7.267933,7.267933
1156222,78546729,Johnson & Johnson,Official Shop,1333628422,2227883317,2019-05-19,1,0.043006,0.043006
1258380,111139273,Yves Rocher,Official Shop,1360665258,1998072190,2019-05-25,1,0.387050,0.387050
175737,78546729,Johnson & Johnson,Official Shop,1331151102,1368300954,2019-05-19,1,3.827491,3.827491
888025,89960894,P&G,Official Shop,1319783625,2052063976,2019-05-16,2,6.308910,12.617820


In [258]:
for_submission = pd.DataFrame({})

for_submission['Index'] = range(1, len(all_brands) + 1)
for_submission['Answers'] = ''

for idx, brand in enumerate(all_brands):
    result = final_df[final_df['brand'] == brand].groupby('itemid')['gross_sales_revenue'].sum().rename('gross_sales').sort_values(ascending=False)[:3].reset_index()
    if len(result['itemid']) > 0:
        for_submission.at[idx, 'Answers'] += (str(brand) + ', ')
        for itemid in result['itemid']:
            for_submission.at[idx, 'Answers'] += (str(itemid) + ', ')
    else:
        for_submission.at[idx, 'Answers'] += (str(brand) + ', N.A,,')
        
for_submission['Answers'] = for_submission['Answers'].apply(lambda x: x[:-2])

In [259]:
for_submission.to_csv('submission1.csv', index=False)